In [ ]:
# !pip install datasets==3.6.0
from huggingface_hub import login
login("")

In [2]:
from datasets import load_dataset, Dataset, Audio, concatenate_datasets, DatasetDict

In [3]:
ds = load_dataset("Elormiden/rik_atn_global", streaming=True)

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

In [4]:
from tqdm import tqdm

In [5]:
ds

IterableDatasetDict({
    train: IterableDataset({
        features: ['audio'],
        num_shards: 21
    })
})

In [6]:
from datasets import DatasetDict, Dataset
from tqdm import tqdm

def chunk_audio_dataset(ds: DatasetDict, chunk_duration: float = 7.0) -> DatasetDict:
    def chunk_generator():
        for example in tqdm(ds['train'], desc="Processing audio files"):
            audio_array = example['audio']['array']
            sampling_rate = example['audio']['sampling_rate']
            chunk_samples = int(chunk_duration * sampling_rate)

            for i in range(0, len(audio_array), chunk_samples):
                chunk = audio_array[i:i + chunk_samples]
                if len(chunk) == chunk_samples:
                    yield {'audio': {'array': chunk, 'sampling_rate': sampling_rate}}

    return DatasetDict({'train': Dataset.from_generator(chunk_generator)})

In [7]:
chunked_ds = chunk_audio_dataset(ds)

Generating train split: 0 examples [00:00, ? examples/s]


Processing audio files: 0it [00:00, ?it/s]
Processing audio files: 1it [00:31, 31.04s/it]
Processing audio files: 2it [00:38, 17.28s/it]
Processing audio files: 3it [00:48, 13.74s/it]
Processing audio files: 4it [00:54, 10.98s/it]
Processing audio files: 5it [01:03, 10.11s/it]
Processing audio files: 6it [01:13,  9.98s/it]
Processing audio files: 7it [01:58, 21.61s/it]
Processing audio files: 8it [02:07, 17.36s/it]
Processing audio files: 9it [02:17, 15.27s/it]
Processing audio files: 10it [02:24, 12.64s/it]
Processing audio files: 11it [02:38, 13.00s/it]
Processing audio files: 12it [02:43, 10.69s/it]
Processing audio files: 13it [03:24, 19.93s/it]
Processing audio files: 14it [03:30, 15.45s/it]
Processing audio files: 15it [03:39, 13.66s/it]
Processing audio files: 16it [03:46, 11.50s/it]
Processing audio files: 17it [03:52,  9.86s/it]
Processing audio files: 18it [04:00,  9.40s/it]
Processing audio files: 19it [04:38, 17.92s/it]
Processing audio files: 20it [04:45, 14.79s/it]
Proce

Loading dataset shards:   0%|          | 0/66 [00:00<?, ?it/s]

In [15]:
chunked_ds['train']['audio'][3]

KeyboardInterrupt: 

In [17]:
chunked_ds.cast_column("audio", Audio(sampling_rate=16000))

KeyboardInterrupt: 

In [16]:
chunked_ds.push_to_hub("Elormiden/rik-atn-global-cut-one-third")

Uploading the dataset shards:   0%|          | 0/118 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/380M [00:00<?, ?B/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/379M [00:00<?, ?B/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/379M [00:00<?, ?B/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/379M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [18]:
from datasets import DatasetDict, Dataset
from itertools import islice
import random

def sample_and_shuffle_dataset(ds: DatasetDict, fraction: float = 1/3, seed: int = 42) -> DatasetDict:
    random.seed(seed)
    # Collect rows from IterableDataset
    rows = list(ds['train'])  # Convert to list (loads into memory, use with caution)
    total_rows = len(rows)
    sample_size = int(total_rows * fraction)
    # Shuffle and sample
    random.shuffle(rows)
    sampled_rows = rows[:sample_size]
    return DatasetDict({'train': Dataset.from_list(sampled_rows)})

In [ ]:
# Assuming ds is your DatasetDict
new_ds = sample_and_shuffle_dataset(chunked_ds)
print(new_ds)

In [ ]:
new_ds.push_to_hub("atn-rik=global=cut-1/3")